In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

In [2]:
filename = "imdb.json"

class IMDbSpider(scrapy.Spider):
    name = "imdb_movies"
    start_urls = ['https://www.imdb.com/chart/top']

    def parse(self, response):

        for movie in response.css(".ipc-metadata-list-summary-item"):
            # Title and rank
            title_and_rank = movie.css("h3.ipc-title__text::text").get()
            rank, title = (title_and_rank.split(". ", 1) if title_and_rank else (None, None))

            # Movie URL
            url = movie.css("a.ipc-title-link-wrapper::attr(href)").get()
            full_url = response.urljoin(url) if url else None

            # Rating
            rating = movie.xpath('.//span[contains(@class, "ipc-rating-star--rating")]/text()').get()

            # Votes
            votes = movie.xpath('.//span[@class="ipc-rating-star--voteCount"]//text()').getall()
            votes = votes[1].strip('"()')

            # Yield the result
            yield {
                "rank": rank,
                "title": title,
                "url": full_url,
                "rating": rating,
                "votes": votes,
            }


process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    },
})

process.crawl(IMDbSpider)
process.start()


2025-02-21 21:39:37 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2025-02-21 21:39:37 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.13.1, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.0.15 3 Sep 2024), cryptography 43.0.0, Platform Windows-11-10.0.26100-SP0
2025-02-21 21:39:37 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-21 21:39:37 [py.warnings] WARNING: c:\Users\darla\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docu

In [3]:
df = pd.read_json('src/imdb.json')
print(df.head())

ValueError: Trailing data